In [20]:
def read_stock_data(data_home, data_type, venue, year, month, day, ticker, is_filter):
    '''
    从sever中读取一只股票一天的数据
    data_home:数据所在folder
    data_type:类型
    venue:交易所
    is_filter:是否进行filter操作
    '''

    path = str(data_home) + '/' + str(data_type) + '/' + str(venue) + '/' + str(year) + '/' + str(
        month) + '/' + str(day) + '/' + str(ticker) + '.' + str(venue)  # 读数据的路径

    if (os.path.exists(path)):
        data0 = open(path, 'r')
        data1 = pd.read_csv(StringIO(data0.read()))
        data = data1.loc[:, ['time', 'volume', 'current','a1_v','a1_p','b1_v','b1_p',]]
        diff_df = data.loc[:, ['time', 'volume']].diff()  # 差分，求出每次交易的交易量、交易额
        data.iloc[1:len(data['volume']), 1] = diff_df.iloc[1:len(data['volume']), 1] # 第0个为NaN,从第一个代替原数据的volume
        data['current'] = data['current']/10000
        data['spread'] = (data['a1_p'] - data['b1_p'])/(data['a1_p'] + data['b1_p'])  # 计算 spread
        if is_filter == 0:  # 不filter数据的时候
            return data
        else:
            quantile = np.percentile(data['volume'], 99.9)  # 计算volume99%分位数
            data = data[data['volume'] <= quantile]         # 删除大于99.9%分位数的数据
            return data
    else:
        path = str(data_home) + '/' + str(data_type) + '/' + str(venue) + '/' + str(year) + '/' + str(
            month) + '/' + str(day) + '/' + str(ticker) + '.' + str(venue) + '.gz'
        with gzip.open(path, 'rb') as gf:
            data1 = pd.read_csv(gf)
        data = data1.loc[:, ['time', 'volume','current', 'a1_v','b1_v']]
        diff_df = data.loc[:, ['time', 'volume']].diff()  # 差分，求出每次交易的交易量、交易额
        data.iloc[1:len(data['volume']), 1] = diff_df.iloc[1:len(data['volume']), 1] # 第0个为NaN,从第一个代替原数据的volume
        data['current'] = data['current']/10000
        if is_filter == 0:  # 不filter数据的时候
            return data
        else:
            quantile = np.percentile(data['volume'], 99.9)  # 计算volume99%分位数
            data = data[data['volume'] <= quantile]  # 删除大于99.9%分位数的数据
            return data

In [21]:
def read_stock_data_all(data_home, data_type, venue, start_date, end_date, ticker, is_filter):
    '''
    读取一只股票所有日期的数据，from start_date to end_date
    '''

    data_concat = pd.DataFrame(columns=['date', 'daily_volume'])

    start_date1 = datetime.datetime.strptime(start_date, '%Y-%m-%d')
    end_date1 = datetime.datetime.strptime(end_date, '%Y-%m-%d')
    interval_day = (end_date1-start_date1).days

    ##遍历日期

    for i in range(interval_day+1):
        date = datetime.datetime.strptime(start_date,'%Y-%m-%d') + datetime.timedelta(days=i)

        date2 = datetime.datetime.strftime(date, '%Y-%m-%d')
        year = date2[0:4]
        month = date2[5:7]
        day = date2[8:10]

        if len(str(month)) < 2:
            month = str(0) + str(month)
        else:
            month = str(month)
        if len(str(day)) < 2:
            day = str(0) + str(day)
        else:
            day = str(day)

        dirs = str(data_home) + '/' + str(data_type) + '/' + str(venue) + '/' + str(year) + '/' + str(
                    month) + '/' + str(day) + '/'


        if not (os.path.exists(dirs)):
            continue
        else:
            print(year, month, day)
            data = get_df(data_home, data_type, venue, year, month, day, ticker, is_filter)
            frames = [data_concat, data]
            data_concat = pd.concat(frames)  # 将一只股票多天的数据合并到一个数据框里

    return data_concat.reset_index(drop=True)  # 返回合并后的数据框并重新设置下标



In [22]:
def trans_date(date):  # transform date to year-month-day
    dates = []
    for i in range(len(date)):
        year = str(date[i])[0:4]
        month = str(date[i])[4:6]
        day = str(date[i])[6:8]
        date_std = datetime.date(int(year), int(month), int(day)).isoformat()
        dates.append(date_std)
    return dates


def trans_time(time):  # transform time to hour:minute:second
    times = []
    for i in range(len(time)):
        hour = str(time[i])[8:10]
        minute = str(time[i])[10:12]
        second = str(time[i])[12:14]
        time_std = datetime.time(int(hour), int(minute), int(second)).isoformat()
        times.append(time_std)
    return times

In [23]:
def get_df(data_home, data_type, venue, year, month, day,ticker, is_filter=0):
    stock_data = read_stock_data(data_home, data_type, venue, year, month, day, ticker, is_filter=0)
    stock_data = stock_data.reset_index(drop=True)
    transdate = trans_date(stock_data['time'])
    transtime = trans_time(stock_data['time'])
    stock_data.loc[:, 'date'] = transdate  # replace the original data by transformed data
    stock_data.loc[:, 'timet'] = transtime
    return stock_data

In [25]:
import gzip
import os
import pandas as pd
from io import StringIO
import numpy as np
import datetime
import dateutil.parser as parser
import datetime
data_home = '/volume1/sinoalgo/data/sinoalgo/JQMarketData'
data_type = 'STOCK'
venue= 'XSHE'
ticker = '000725'
start_date = "2020-09-01"
end_date = "2021-06-30"
data = read_stock_data_all(data_home, data_types, venues, start_date, end_date, tickers, 0)
data.to_csv("stock_price",index=False)